In [17]:
from ASmartis import * 
from ipywidgets import HBox

class Kondensator(ASmartis):
    
    def __init__(self,parameterList):
        self.voltage = parameterList[0]
        self.distance = parameterList[1]
        self.area = parameterList[2]
        self.dielectric = parameterList[3]
        self.outputType = parameterList[4]
        self.plateWidth = 20
        
        super().__init__(parameterList)  
        self.reset()
        self.updateTime = 0.08
    def createButtons(self):        
        self.buttonsObject = Buttons(self)
        
        self.buttonsObject.newReset("b_reset", 40,40) 
        self.buttonsObject.newFloatTextbox("t_voltage", 1, r"$U_{2}$")
        self.buttonsObject.newFloatSlider("s_voltage", 1,2,0.05, r"$U$")
        self.buttonsObject.newIntSlider("s_distance", 100,200,10, r"$d$")
        self.buttonsObject.newIntSlider("s_area", 100,300,100, r"$A$")
        self.buttonsObject.newIntSlider("s_dielectric", 1,4,0, r"$\varepsilon_{r}$")
        self.buttonsObject.newRadioButtons("b_fieldArea",["homogen", 
            "geschichtet, seriell","geschichtet, parallel"], "Dielektrikum:") 
        self.buttonsObject.newRadioButtons("b_fieldType", ["E-Feld","D-Feld"], "Feldlinien:")
                
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
        
        
            
    def createChart(self):        
        self.layers = ["background","capacitor","dielectricum","field","protons","electrons"]    
        self.chartWidth = 400
        self.chartHeight = 425
        self.chartObject = Chart(self.layers, self.chartWidth, self.chartHeight)
                
        self.x_capacitorCenter = 200
        self.y_capacitorCenter = 180
        
        currentLayer = "background"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#000000")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffffff")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        
        currentLayer = "capacitor"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#000000")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#f0f0f0")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        
        currentLayer = "dielectricum"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#99ff99")        
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",1)
        
        currentLayer = "field"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#e8082c") 
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#e8082c")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",1)
        
                
    def visualize(self):          
        chart = self.chartObject.getChart()
        buttonsDict = self.buttonsObject.getButtons()
          
        display(HBox([chart,buttonsDict["b_reset"]]))
                # Widgets je nach Diagrammtyp darstellen
        if self.outputType == 0:   # Einfluss der Geometrie
            display(HBox([buttonsDict["s_voltage"],widgets.Label(value="V")]))
            display(HBox([buttonsDict["s_distance"],widgets.Label(value="mm")]))
            display(HBox([buttonsDict["s_area"],widgets.Label(value="cm²")]))
            
        elif self.outputType == 1: # Einfluss des Dielektrikums
            display(HBox([buttonsDict["s_dielectric"],widgets.Label(value=" ")]))
            display(HBox([buttonsDict["b_fieldArea"],buttonsDict["b_fieldType"]]))
            
        elif self.outputType == 2: # Isolierte Anschlussklemmen
            display(HBox([buttonsDict["t_voltage"],widgets.Label(value="V")]))
            display(HBox([buttonsDict["s_distance"],widgets.Label(value="mm")]))
            display(HBox([buttonsDict["s_dielectric"],widgets.Label(value=" ")]))
            display(HBox([buttonsDict["b_fieldType"]]))
    
        else: #Alle Buttons werden angezeigt
            display(HBox([buttonsDict["s_voltage"],buttonsDict["t_voltage"]]))
            display(HBox([buttonsDict["s_distance"],buttonsDict["s_dielectric"]]))
            display(HBox([buttonsDict["s_area"]]))
            display(HBox([buttonsDict["b_fieldArea"],buttonsDict["b_fieldType"]]))
        
        
        
    def update(self):   
        if self.buttonsStatesDict["b_reset"]:
            self.reset() 
        else:
            if self.buttonsStatesDict["s_voltage"] != self.oldButtonsStatesDict["s_voltage"]:
                self.draw_chargesAndField()
                self.chartObject.update("field","protons","electrons")
            elif (self.buttonsStatesDict["s_distance"] != self.oldButtonsStatesDict["s_distance"] or
                self.buttonsStatesDict["s_area"] != self.oldButtonsStatesDict["s_area"]):
                self.draw_background()
                self.draw_capacitor()
                self.draw_dielectricum()
                self.draw_chargesAndField()    
                self.chartObject.update("background","capacitor","dielectricum","field","protons","electrons")
                
            elif (self.buttonsStatesDict["s_dielectric"] != self.oldButtonsStatesDict["s_dielectric"]or
                self.buttonsStatesDict["b_fieldArea"] != self.oldButtonsStatesDict["b_fieldArea"] or
                self.buttonsStatesDict["b_fieldType"] != self.oldButtonsStatesDict["b_fieldType"]):
                self.draw_dielectricum()
                self.draw_chargesAndField() 
                self.chartObject.update("dielectricum","field","protons","electrons")
                
                
    def reset(self):
        self.buttonsObject.changeState("s_voltage",self.voltage)
        self.buttonsObject.changeState("s_distance",self.distance)
        self.buttonsObject.changeState("s_area",self.area)
        self.buttonsObject.changeState("s_dielectric",self.dielectric)
           
        self.draw_background()
        self.draw_capacitor()
        self.draw_dielectricum()
        self.draw_chargesAndField()    
        self.chartObject.update("background","capacitor","dielectricum","field","protons","electrons")
        self.buttonsObject.changeState("b_reset",False)
        
    def draw_background(self):
        
        currentDistance = self.buttonsStatesDict["s_distance"]+2*self.plateWidth
        x_startCapacitor = 25+(350-currentDistance)/2
        x_endCapacitor = x_startCapacitor+currentDistance
        
        
        currentLayer = "background"
        self.chartObject.clearLayer(currentLayer)
        self.chartObject.drawLine(currentLayer,25,375,375,375) 
        self.chartObject.drawCircle(currentLayer,200,375,24)
        if self.outputType == 2:
            self.chartObject.drawLine(currentLayer,200,385,208,365)
            self.chartObject.drawLine(currentLayer,200,385,192,365)
        else:
            self.chartObject.drawLine(currentLayer,25,375,375,375)
            
        self.chartObject.drawLine(currentLayer,25,180,25,375)
        self.chartObject.drawLine(currentLayer,375,180,375,375)
        
        self.chartObject.drawLine(currentLayer,25,180,x_startCapacitor,180)
        self.chartObject.drawLine(currentLayer,x_endCapacitor,180,375,180)
        
                
    
    def draw_capacitor(self):
        currentLayer = "capacitor"
        self.chartObject.clearLayer(currentLayer)
        #Maße einer Platte
        width = self.plateWidth
        height = self.buttonsStatesDict["s_area"]
        currentDistance = self.buttonsStatesDict["s_distance"]
        #Rechte Platte
        x_cap = self.x_capacitorCenter-width-currentDistance/2
        y_cap = self.y_capacitorCenter-height/2        
        self.chartObject.drawRectangle(currentLayer,x_cap,y_cap,width,height)
        #Linke Platte
        x_cap = self.x_capacitorCenter+width+currentDistance/2
        y_cap = self.y_capacitorCenter-height/2          
        self.chartObject.drawRectangle(currentLayer,x_cap,y_cap,-width,height)
        
        
        
    def draw_dielectricum(self):
        currentLayer = "dielectricum"
        self.chartObject.clearLayer(currentLayer)
        if self.buttonsStatesDict["b_fieldArea"] == "homogen":
            height = self.buttonsStatesDict["s_area"]
            width = self.buttonsStatesDict["s_distance"]
            x_start = self.x_capacitorCenter-width/2
            y_start = self.y_capacitorCenter-height/2           
            
        elif self.buttonsStatesDict["b_fieldArea"] == "geschichtet, seriell":
            height = self.buttonsStatesDict["s_area"]
            width = self.buttonsStatesDict["s_distance"]/2
            x_start = self.x_capacitorCenter
            y_start = self.y_capacitorCenter-height/2 

        elif self.buttonsStatesDict["b_fieldArea"] == "geschichtet, parallel":
            height = self.buttonsStatesDict["s_area"]/2
            width = self.buttonsStatesDict["s_distance"]
            x_start = self.x_capacitorCenter-width/2
            y_start = self.y_capacitorCenter
        else:
            pass
        
        
        self.chartObject.drawParallelLines(currentLayer,x_start,y_start,
                width, height, self.buttonsStatesDict["s_dielectric"]*10, "diagonal")
    
    def draw_chargesAndField(self):
        
        distance = self.buttonsStatesDict["s_distance"] 
        area = self.buttonsStatesDict["s_area"]
        dielectricum = self.buttonsStatesDict["s_dielectric"]
        fieldArea = self.buttonsStatesDict["b_fieldArea"] 
        fieldType = self.buttonsStatesDict["b_fieldType"]
        #voltage = self.buttonsStatesDict["s_voltage"]
        
        
        #Spannung/Ladung bestimmen
        if self.outputType == 0:
            voltage = self.buttonsStatesDict["s_voltage"]*3
            #Abstand zwischen Ladungen
            chargesDistance = distance/(voltage*dielectricum)
            #Anzahl der Ladungen
            n_charges = int(area/chargesDistance)
            #Bereich Ladungen
            chargesArea = n_charges*chargesDistance
            
        elif self.outputType == 1:  
            voltage = self.buttonsStatesDict["s_voltage"]
            #Abstand zwischen Ladungen
            chargesDistance = distance/(voltage*dielectricum)
            #Anzahl der Ladungen
            n_charges = int(area/chargesDistance)
            
        else:
            n_charges = 10 
            voltage = n_charges*distance/(dielectricum*area)/10
            chargesDistance = int(area/n_charges)
            self.buttonsObject.changeState("t_voltage",voltage)
        
        #Bereich Ladungen
        chargesArea = n_charges*chargesDistance
        
        layerProtons = "protons"
        layerElectrons = "electrons"
        layerField = "field"
        self.chartObject.clearLayer(layerProtons)
        self.chartObject.clearLayer(layerElectrons)
        self.chartObject.clearLayer(layerField)
        radiusCharges = 6
        
        x_protons = self.x_capacitorCenter-distance/2-radiusCharges
        x_electrons = self.x_capacitorCenter+distance/2+radiusCharges
        x_arrow = self.x_capacitorCenter-distance/2
        
        n = n_charges
        y_start = (self.y_capacitorCenter-chargesArea/2)+chargesDistance/2
        
        if fieldArea == "homogen":
            for i in range(n):
                self.chartObject.drawProton(layerProtons,x_protons,y_start+i*chargesDistance,radiusCharges)
                self.chartObject.drawElectron(layerElectrons,x_electrons,y_start+i*chargesDistance,radiusCharges)
                

                if fieldType == "D-Feld":
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                elif fieldType == "E-Feld" and i%dielectricum==0:
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                                    
        elif fieldArea == "geschichtet, seriell":
            for i in range(n):
                self.chartObject.drawProton(layerProtons,x_protons,y_start+i*chargesDistance,radiusCharges)
                self.chartObject.drawElectron(layerElectrons,x_electrons,y_start+i*chargesDistance,radiusCharges)
                if fieldType == "D-Feld":
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                elif fieldType == "E-Feld":
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance/2,y_start+i*chargesDistance,10,6)
                    if i%dielectricum==0:
                        self.chartObject.drawArrow(layerField,x_arrow+distance/2,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                
        elif fieldArea == "geschichtet, parallel":
            for i in range(0,int(abs(n/2)),2):
                self.chartObject.drawProton(layerProtons,x_protons,y_start+i*chargesDistance,radiusCharges)
                self.chartObject.drawElectron(layerElectrons,x_electrons,y_start+i*chargesDistance,radiusCharges)
                if fieldType == "D-Feld":
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                elif fieldType == "E-Feld" and i%dielectricum==0:
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
        
            for i in range(int(abs(n/2)),int(n),1):
                self.chartObject.drawProton(layerProtons,x_protons,y_start+i*chargesDistance,radiusCharges)
                self.chartObject.drawElectron(layerElectrons,x_electrons,y_start+i*chargesDistance,radiusCharges)
                if fieldType == "D-Feld":
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
                elif fieldType == "E-Feld" and i%dielectricum==0:
                    self.chartObject.drawArrow(layerField,x_arrow,y_start+i*chargesDistance,x_arrow+distance,y_start+i*chargesDistance,10,6)
        
        else:
            pass

In [18]:
if __name__ == '__main__':
    e = Kondensator([1, 100, 200, 1, 0])
    e.draw_chargesAndField()
    e.visualize()
    e.start()

In [19]:
if __name__ == '__main__':
    e = Kondensator([1, 90, 200, 1, 1])
    e.draw_chargesAndField()
    
    e.visualize()
    e.start()

In [20]:
if __name__ == '__main__':
    e = Kondensator([1, 100, 200, 1, 2])
    e.draw_chargesAndField()
    e.visualize()
    e.start()